# Math foundation

## Ray - surface intersection in 3D

### Definitions

A 3D ray is represented parametrically as the set of points $P + tV$ with $t \in \mathbb{R}$, where:

* $P$ is an origin point
* $V$ is a unit direction vector

A 3D surface is defined with a function $F$ of the form: $F(x,y,z) = 0$.

To compute the intersection of a ray with a surface, we are looking for the unknown $t$ such that a point is on the surface and on the ray:

$$
F(P + tV) = 0
$$

We call the quantity on the left side $Q(t)$, and finding the intersections is finding the roots of $Q$.

### Newton's method

In reality there may be multiple (or no solution). In practice we are going to solve it iteratively so this issue is minimized.

We can solve the intersection equation using Newton's method with the update step:

$$
t_{n+1} = t_n - \frac{Q(t_n)}{Q'(t_n)}
$$

Developing with the multivariate chain rule we get:

$$
t_{n+1} = t_n - \frac{F(P + t_nV)}{V \cdot \nabla F(P + t_n V)}
$$

where "$\cdot$" in the denominator is the dot product, and $\nabla F$ is the gradient of F:

$$
\nabla F = \left(\frac{dF}{dx}, \frac{dF}{dy}, \frac{dF}{dz}\right)
$$

A 3D surface must define functions to compute $F$ and $\nabla F$.

$Q'$ is zero, and the update step is undefined, when the ray and the surface derivative are parallel, or when the norm of the derivative is zero (which should never be the case).

## Surface of revolution

### 2D shape definition

A 2D shape is defined in the $(x, r)$ plane with the equation: $x = f(r)$. X is the optical axis, and R is the meridional axis, aka the axis perpendiular to the optical axis.

Additionally, for working with lenses, we always have $f(0) = 0$ (the curve crosses the origin) and $f'(0) = 0$ (the curve is vertical at the origin).

Therefore the equation for the 2D curve is in implicit form: $f(r) - x = 0$.

### Rotational symmetry

We want to create the corresponding 3D surface by rotation around the X axis. This means that the R axis from before, is now really the axis of the meridional plane. (A meridional plane is a plane that contains the optical axis X).

So $r$ is the distance from the X axis to any point on the surface. In 3D, we have for any meridional plane $r = \sqrt{y^2 + z^2}$.

The definition of the 3D surface is that the intersection of every meridional plane with it is the 2D surface:

$$
x = f(r)
$$

Therefore, the equation for the 3D surface of revolution for the $f$ curve is:

$$
F(x,y,z) = f(\sqrt{y^2 + z^2}) - x = 0
$$

## Generic form of $\nabla F$ for surfaces of revolution

We have:

$$
F(x, y, z) = f \left( \sqrt{y^2 + z^2} \right) - x
$$

Therefore:

$$
\frac{dF}{dx} = -1
$$
$$
\frac{dF}{dy} = \frac{y}{\sqrt{y^2 + z^2}} f' \left( \sqrt{y^2 + z^2} \right)
$$
$$
\frac{dF}{dz} = \frac{z}{\sqrt{y^2 + z^2}} f' \left( \sqrt{y^2 + z^2} \right)
$$

However, for some curves $f$ this expression simplifies a lot and therefore shapes can provide an optimized version of $\nabla F(x, y, z)$, or even $\nabla F(x,y,z) \cdot V$.

